In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [5]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import time

In [15]:


# Descarga de data
# ==============================================================================
url = ('https://raw.githubusercontent.com/JoaquinAmatRodrigo/Estadistica-machine-learning-python/' +
       'master/data/visitas_por_dia_web_cienciadedatos.csv')
data = pd.read_csv(url, sep=',')
data['date'] = pd.to_datetime(data['date'], format='%d/%m/%y')
data = data.set_index('date')
data = data.asfreq('D')
data = data.sort_index()
len(data)

421

In [21]:
# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 7 
             )

In [22]:
refits = list(range(2, 4))
refits = [False, True] + refits
results = []
metrics = []

for refit in refits:
    tic = time.perf_counter()

    metric, predictions = backtesting_forecaster(
                            forecaster            = forecaster,
                            y                     = data['users'],
                            steps                 = 7,
                            metric                = 'mean_absolute_error',
                            initial_train_size    = 200,
                            fixed_train_size      = False,
                            refit                 = refit,
                            verbose               = False,
                            show_progress         = False  
                        )

    toc = time.perf_counter()
    results.append(toc-tic)
    metrics.append(metric)

results = pd.DataFrame(
              data =  {'seconds': results, 'metric': metrics},
              index = refits
          )

results.sort_values(by=['seconds'])

,seconds,metric
False,1.159208,269.060724
3,2.965323,255.188326
2,3.885899,262.981086
True,7.013977,255.554389
